In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as pchs
import numpy as np
import imageio as iio
import os


In [ ]:

#globals

# set up figure x and y size
width, height = 150, 150
# set up loop duration in seconds
duration = 1
#setup number of frames at 24 fps
frames = (duration * 24)


filenames = []

# create a translation matrix
# x and y are the translation in the x and y direction
def tranMatrix(x,y):
    return np.array([[1,0,x],
                     [0,1,y],
                     [0,0,1]])

# create a rotation matrix
# angle is the rotation in radians
def rotMatrix(angle):
    return np.array([[np.cos(angle), -np.sin(angle), 0],
                     [np.sin(angle),  np.cos(angle), 0],
                     [            0,              0, 1]])

# create a scaling matrix
# x and y are the scaling in the x and y direction
def scaleMatrix(x,y):
    return np.array([[x,0,0],
                     [0,y,0],
                     [0,0,1]])

# apply a matrix to a polygon
# mat is the matrix to apply, pol is the polygon to apply it to
def applyMatrix(mat,pol):
    verts = []
    for i in range(pol.shape[0]):
        verts.append(mat@pol[i])
    return np.array(verts)

# create a box shape
# s is the size of the box
def box(s):
    return np.array([[-s/2,-s/2, 1],
                     [-s/2, s/2, 1],
                     [ s/2, s/2, 1],
                     [ s/2,-s/2, 1]])

# add a shape to the plot
def addShape(ax, verts):
    verts = np.delete(verts,2,1)
    square = pchs.Polygon(verts) 
    ax.add_patch(square)


for i in range(frames):
    # setup figure and axis
    fig, ax = plt.subplots()
    ax.set_xlim(0,width)
    ax.set_ylim(0,height)
    ax.set_aspect('equal')
    
    # frame time
    ft = i/frames

    # create a box shape
    verts = box(20)

    # translation to center
    center = tranMatrix(width/2,height/2)
    # rotation anticlockwise at 1hz
    fullRot = rotMatrix(2*np.pi*ft)

    transform = tranMatrix(25, 0)
    
    # tranform to rotate around the center of the screen with a radius of 25
    transform2 = center@fullRot
    transform = transform2@transform
    
    # apply the transform to the box shape
    verts = applyMatrix(transform, verts)
    addShape(ax,verts)

    # create a second box shape
    verts = box(10)
    
    # transform to rotate anticlockwise around the center of the screen
    # at 2hz 180 degrees offset from previous box and with a radius of 45
    transform = tranMatrix(-45, 0)
    transform2 = transform2@fullRot
    transform = transform2@transform
    
    # apply the transform to the second box shape
    verts = applyMatrix(transform, verts)
    addShape(ax, verts)

    # create a third box shape
    verts = box(10)

    # transform to rotate clockwise at the center of the screen
    # at 2hz with the x axis scaled 3 times bigger
    transform = scaleMatrix(3, 1)
    transform = rotMatrix(-4*np.pi*ft)@transform
    transform = center@transform

    # apply the transform to the third box shape
    verts = applyMatrix(transform, verts)
    addShape(ax,verts)
    
    #create temprary images
    filename = f'temp_{i}.png'
    filenames.append(filename)

    plt.savefig(filename)
    plt.close()

# Create a GIF from temporary images
with iio.get_writer('example.gif', mode='I', duration=duration, loop=0) as writer:
    for filename in filenames:
        image = iio.imread(filename)
        writer.append_data(image)

# Cleanup temporary images
for filename in filenames:
    os.remove(filename)
